In [7]:
from maap.maap import MAAP
maap = MAAP()

# --- REQUIRED: set these to your registered algorithm ---
ALGO_ID      = "run_zonal_stats_vrt_list"   # e.g., what you picked in the UI
ALGO_VERSION = "v2025_10_08"                # the version tag/name from registration
WORKER_TYPE  = "maap-dps-worker-8gb"        # or a larger queue if needed

# A friendly name for this run (used in output paths under /projects/<bucket>/dps_output/)
IDENTIFIER = "ZONAL_HLS_BRL_ATLANTIC_2025"
RUN_NAME   = IDENTIFIER
USER       = "rodrigo.leite"                # purely for your own tracking table


In [8]:
# ---- SINGLE JOB (set one tile + one year) ----
ref_tile_list = [89]        # <- your tile number(s)
ref_year_list = [2022]       # <- your reference vegetation year(s)

# Sanity check: lists should be same length if paired; otherwise we’ll cartesian-product below.
assert len(ref_tile_list) == len(ref_year_list)


In [9]:
import datetime
import pandas as pd
from pathlib import Path

DPS_SUBMISSION_RESULTS_DIR = "/projects/my-public-bucket/dps_submission_results"
Path(DPS_SUBMISSION_RESULTS_DIR).mkdir(parents=True, exist_ok=True)

# Constant pieces
DIR_IMG = "/projects/my-private-bucket/HLS-1DCNN-AGB/data/tif/HLS_composites/monthly/br_af_grid60km_prj_evi2_max/vrt/"
ZONE_ID_COL = "zone"
BUFFER_METERS = 0

# IMPORTANT: pass EXCLUDE_BANDS as a space-separated string so run.sh expands it to multiple args
EXCLUDE_BANDS_STR = "ValidMask.vrt count.vrt yearDate.vrt JulianDate.vrt"

submit_results_df_list = []
now_stamp = pd.Timestamp.now().strftime("%Y%m%d%H%M")


In [10]:
for i, (ref_tile, ref_year) in enumerate(zip(ref_tile_list, ref_year_list), start=1):

    gpkg_points_path = f"/projects/my-private-bucket/HLS-1DCNN-AGB/data/shp/gedi/test/l4a_t90km_t{ref_tile}_veg{ref_year}_outrm.gpkg"
    output_gpkg_zonalstats_fn = f"/projects/my-private-bucket/HLS-1DCNN-AGB/data/shp/gedi/l4a_t90km_t{ref_tile}_veg{ref_year}_outrm_zonal_HLS.gpkg"

    in_params_dict = {
        # names must match what run.sh reads and forwards to your Python CLI
        "GPKG": gpkg_points_path,
        "DIR_IMG": DIR_IMG,
        "OUTPUT": output_gpkg_zonalstats_fn,
        "ZONE_ID_COL": ZONE_ID_COL,
        "BUFFER_M": str(BUFFER_METERS),      # strings are safest for env vars
        "EXCLUDE_BANDS": EXCLUDE_BANDS_STR,  # space-separated list (run.sh will split)
    }

    submit_result = maap.submitJob(
        identifier=IDENTIFIER,
        algo_id=ALGO_ID,
        version=ALGO_VERSION,
        queue=WORKER_TYPE,
        **in_params_dict
    )

    submit_results_df_list.append(
        pd.DataFrame({
            "dps_num":        [i],
            "ref_tile":       [ref_tile],
            "ref_year":       [ref_year],
            "submit_time":    [datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")],
            "algo_id":        [ALGO_ID],
            "algo_version":   [ALGO_VERSION],
            "user":           [USER],
            "worker_type":    [WORKER_TYPE],
            "job_id":         [submit_result.id],
            "submit_status":  [submit_result.status],
            "run_name":       [RUN_NAME],
            "gpkg":           [gpkg_points_path],
            "dir_img":        [DIR_IMG],
            "output":         [output_gpkg_zonalstats_fn],
            "zone_id_col":    [ZONE_ID_COL],
            "buffer_m":       [BUFFER_METERS],
            "exclude_bands":  [EXCLUDE_BANDS_STR],
        })
    )

    print(f"[{i}/{len(ref_tile_list)}] job_id={submit_result.id} status={submit_result.status} tile={ref_tile} year={ref_year}")


[1/1] job_id= status=failed tile=89 year=2022
